# Cargar BD

## Extraer paths a cargar

In [1]:
import os
import pandas as pd
from utils.aggregation_functions import (rellenar_etas, agrupar_descargas,
                                         estimar_demurrage, formato_BD)
from utils.extraction_functions import (extraer_bts, extraer_descargas, extraer_tiempos_de_viaje,
                   extraer_planificacion, extraer_programas, extraer_nueva_ficha,
                   extraer_productos_plantas, extraer_reporte_tankers, DF_PLANTAS)

PATH_DISTANCIAS = "Distancias entre puertos.xlsx"
PATH_NUEVA_FICHA = "C:\\Users\\jaubele.ap\\Desktop\\Estimación Semanal\\Nueva Ficha Información de Buques.xlsx"
CARPETA_PROGRAMACIONES = "C:\\Users\\jaubele.ap\\Desktop\\Estimación Semanal\\Programaciones"
PATHS_PROGRAMACIONES = [os.path.join(CARPETA_PROGRAMACIONES, archivo) for archivo in os.listdir(CARPETA_PROGRAMACIONES) if archivo.endswith('.xlsx')]

## Crear conexión a BD

In [2]:
from modelo import Base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy_utils import create_database
from utils.loading_functions import (nueva_ficha_psql_format, plantas_psql_format,
                                     create_descargas, update_estimaciones_programas,
                                     create_programacion, get_programacion,
                                     update_programas, BD_URI)

engine = create_engine(BD_URI)

## Crear BD si no existe y subir datos inciales

In [3]:
Base.metadata.drop_all(engine)

In [4]:
# create_database(engine.url) 
Base.metadata.create_all(engine)

df_nueva_ficha_formateado = nueva_ficha_psql_format(extraer_nueva_ficha(PATH_NUEVA_FICHA, "Programación de buques"))
df_nueva_ficha_formateado.to_sql("programas", engine, if_exists="append", index=False)

df_plantas_formateado = plantas_psql_format(DF_PLANTAS)
df_plantas_formateado.to_sql("plantas", engine, if_exists="append", index=False)

9

## Ejecutar APP

In [5]:
def execute_app(PATH_PROGRAMACION, PATH_NUEVA_FICHA, engine):
    df_bts = extraer_bts(PATH_PROGRAMACION, "Buques")
    df_bts_puma_enap = extraer_bts(PATH_PROGRAMACION, "Buques", add_puma=True, add_enap=True)
    df_planificacion = extraer_planificacion(PATH_PROGRAMACION, "Planificación")
    df_descargas = extraer_descargas(df_planificacion, ignore_not_bts=True, df_bts=df_bts)
    df_descargas_puma_enap = extraer_descargas(df_planificacion, ignore_not_bts=False)
    df_programas = extraer_programas(df_planificacion)
    df_productos_plantas = extraer_productos_plantas()
    df_nueva_ficha = extraer_nueva_ficha(PATH_NUEVA_FICHA, "Programación de buques", df_programas=df_programas)
    matriz_de_tiempos = extraer_tiempos_de_viaje("Distancias entre puertos.xlsx", "Datos")

    df_descargas_productos_plantas = df_descargas.merge(df_productos_plantas, on=["Columna"]).drop(columns=["Columna"])
    df_descargas_productos_plantas_puma_enap = df_descargas_puma_enap.merge(df_productos_plantas, on=["Columna"]).drop(columns=["Columna"])
    df_descargas_completo = df_descargas_productos_plantas.merge(df_bts, on=["Abrev."]).drop(columns=["Abrev."])
    df_descargas_completo_puma_enap = df_descargas_productos_plantas_puma_enap.merge(df_bts_puma_enap, on=["Abrev."]).drop(columns=["Abrev."])
    df_descargas_completo = df_descargas_completo[["Fecha", "N° Referencia", "Nombre programa", "Nombre del BT",
                                                "Producto", "Planta", "Ciudad", "Alias", "Volumen"]]
    df_descargas_completo_puma_enap = df_descargas_completo_puma_enap[["Fecha", "N° Referencia", "Nombre programa", "Nombre del BT",
                                                "Producto", "Planta", "Ciudad", "Alias", "Volumen"]]
    df_descargas_agrupadas = agrupar_descargas(df_descargas_completo)

    df_programas_completo = df_programas.merge(df_nueva_ficha, on="N° Referencia", how="left")
    df_programas_completo["Inicio Ventana"] = df_programas_completo["Inicio Ventana Corta"].combine_first(df_programas_completo["Inicio Ventana"])
    df_programas_completo["Fin Ventana"] = df_programas_completo["Fin Ventana Corta"].combine_first(df_programas_completo["Fin Ventana"])
    df_programas_completo["ETA"] = df_programas_completo["ETA"].combine_first(df_programas_completo["ETA Programa"])
    # Comentar si no se desea llenar montos faltantes con el promedio
    # df_programas_completo["MONTO ($/DIA)"] = df_programas_completo["MONTO ($/DIA)"].fillna(df_programas_completo["MONTO ($/DIA)"].mean()).astype(int)
    # Comentar si no se desea llenar montos faltantes con 35.000
    df_programas_completo["MONTO ($/DIA)"] = df_programas_completo["MONTO ($/DIA)"].fillna(35000).astype(int)
    df_programas_completo = df_programas_completo.drop(columns=["Inicio Ventana Corta", "Fin Ventana Corta", "ETA Programa"])

    df_descargas_descartadas = df_descargas_agrupadas[~df_descargas_agrupadas["N° Referencia"].isin(df_programas_completo["N° Referencia"])]

    df_descargas_por_programa = df_descargas_agrupadas.merge(df_programas_completo, on="N° Referencia", how="right")
    df_descargas_por_programa = df_descargas_por_programa[df_descargas_por_programa["Producto"].notna()]
    df_descargas_por_programa["Nombre del BT_x"] = df_descargas_por_programa["Nombre del BT_x"].combine_first(df_descargas_por_programa["Nombre del BT_y"])
    df_descargas_por_programa.rename(columns={"Nombre del BT_x": "Nombre del BT"}, inplace=True)
    df_descargas_por_programa = df_descargas_por_programa.drop(columns=["Nombre del BT_y"])
    df_descargas_por_programa.index = range(0, len(df_descargas_por_programa))
    df_descargas_por_programa["ETA"] = df_descargas_por_programa["ETA"][[True if descarga == 1 else False for descarga in df_descargas_por_programa["N° Descarga"]]]
    df_descargas_por_programa = rellenar_etas(df_descargas_por_programa, matriz_de_tiempos)
    df_estimacion = estimar_demurrage(df_descargas_por_programa)

    FECHA_PROGRAMACION = pd.to_datetime(df_planificacion.loc[13, "B"], format="%d-%m-%Y", errors="coerce")
    df_BD = formato_BD(df_estimacion, df_descargas_completo, FECHA_PROGRAMACION)
    
    print(f"Cargando programación del {FECHA_PROGRAMACION}...")
    with Session(engine) as session:
        programas = update_programas(session, df_programas_completo) # Se actualizan los programas con la nueva ficha + reporte tankers
        programacion = get_programacion(session, FECHA_PROGRAMACION)
        if programacion:
            print(f"La programación del {FECHA_PROGRAMACION} ya existe en la base de datos. Se eliminará para cargar la nueva programación.")
            session.delete(programacion)
            session.commit()
        programacion = create_programacion(session, FECHA_PROGRAMACION) # Se crea la nueva programación
        descargas_descartadas = create_descargas(session, df_descargas_descartadas, programacion, estimacion=False)
        df_descargas_con_estimacion = create_descargas(session, df_estimacion, programacion, estimacion=True)

        df_estimacion_con_año_mes = df_estimacion.merge(df_BD[["CC", "Año", "Mes"]], left_on="N° Referencia", right_on="CC", how="left")
        df_estimacion_programas_con_año_mes = df_estimacion_con_año_mes.drop_duplicates(subset=["N° Referencia"])
        update_estimaciones_programas(session, df_estimacion_programas_con_año_mes)
    print(f"Cargada programación del {FECHA_PROGRAMACION} completa.")

In [6]:
for i, PATH_PROGRAMACION in enumerate(PATHS_PROGRAMACIONES[:-1]):
    print(f"Procesando archivo {i+1} de {len(PATHS_PROGRAMACIONES)}: {PATH_PROGRAMACION}")
    execute_app(PATH_PROGRAMACION, PATH_NUEVA_FICHA, engine)

Procesando archivo 1 de 15: C:\Users\jaubele.ap\Desktop\Estimación Semanal\Programaciones\01-Programacion Descarga Importaciones 08 de Sep.xlsx


c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jaubele.ap\Desktop\Automatizacion-Programacion-BD\utils\aggregation_functions.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current['Ciudad'] = prev['Ciudad']


Cargando programación del 2025-09-08 00:00:00...
Cargada programación del 2025-09-08 00:00:00 completa.
Procesando archivo 2 de 15: C:\Users\jaubele.ap\Desktop\Estimación Semanal\Programaciones\02-Programacion Descarga Importaciones 10 de Sep.xlsx


c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jaubele.ap\Desktop\Automatizacion-Programacion-BD\utils\aggregation_functions.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current['Ciudad'] = prev['Ciudad']


Cargando programación del 2025-09-10 00:00:00...
Cargada programación del 2025-09-10 00:00:00 completa.
Procesando archivo 3 de 15: C:\Users\jaubele.ap\Desktop\Estimación Semanal\Programaciones\03-Programacion Descarga Importaciones 15 de Sep.xlsx


...

Cargando programación del 2025-10-20 00:00:00...
Cargada programación del 2025-10-20 00:00:00 completa.
Procesando archivo 13 de 15: C:\Users\jaubele.ap\Desktop\Estimación Semanal\Programaciones\13-Programacion Descarga Importaciones 27 de Oct.xlsx


c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Cargando programación del 2025-10-27 00:00:00...
Cargada programación del 2025-10-27 00:00:00 completa.
Procesando archivo 14 de 15: C:\Users\jaubele.ap\Desktop\Estimación Semanal\Programaciones\14-Programacion Descarga Importaciones 03 de Nov.xlsx


c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Cargando programación del 2025-11-03 00:00:00...
Cargada programación del 2025-11-03 00:00:00 completa.
